In [ ]:
import cv2
import numpy as np
from scipy.spatial import distance as dist 
import time



#color filters
HSV_RED_MIN = np.array([165, 30, 30])
HSV_RED_MAX = np.array([180, 255, 255])
HSV_ORANGE_MIN = np.array([0, 30, 30])
HSV_ORANGE_MAX = np.array([15, 255, 255])
HSV_GREEN_MIN = np.array([70, 20, 20])
HSV_GREEN_MAX = np.array([90, 200, 200])
HSV_BLUE_MIN = np.array([100, 20, 20])
HSV_BLUE_MAX = np.array([140, 230, 230])

TOLERANCE_HSV = np.array([8, 120, 120])

#change those values with the camera 
RED_BGR = np.array([115, 95, 194])
BLUE_BGR = np.array([255, 0, 0])
GREEN_BGR = np.array([0, 255, 0])
ORANGE_BGR = np.array([0, 128, 255])

#camera and map parameters
CAMERA_WIDTH = 1024
CAMERA_HEIGHT = 768
BIRD_WIDTH = 550
BIRD_HEIGHT = 350
CAMERA_CORNERS = [[0, 0], [CAMERA_WIDTH, 0], [CAMERA_WIDTH, CAMERA_HEIGHT], [0, CAMERA_HEIGHT]]
BIRD_CORNERS = [[0, 0], [BIRD_WIDTH, 0], [BIRD_WIDTH, BIRD_HEIGHT], [0, BIRD_HEIGHT]]
MAP_WIDTH = 1200
MAP_HEIGHT = 900

#matching thresholds
RES_THRESHOLD_BLACK = 0.82
RES_THRESHOLD_RED = 0.4
RES_THRESHOLD_GREEN = 0.6

#binary thresholds
BINARY_THRESHOLD_BLACK = 45
BINARY_THRESHOLD_RED = 27
BINARY_THRESHOLD_GREEN = 27
BINARY_THRESHOLD_ORANGE = 22

OVERLAP_THRESHOLD = 0
THYMIO_WIDTH = 5
TEMP_CENTER_Y = 53
TEMP_CENTER_X = 69

OBSTACLE_MAP_VALUE = -1
BACKGROUND_MAP_VALUE = 0

#templates
template_cross = cv2.imread('/home/victor-pdt/Documents/Mobile_Robotics/template_cross_binary2.jpg')
template_l_thymio = cv2.imread('/home/victor-pdt/Documents/Mobile_Robotics/template_thymio_T.jpg')
template_goal = cv2.imread('/home/victor-pdt/Documents/Mobile_Robotics/template_goal_binary.jpg')

method = eval('cv2.TM_CCORR_NORMED')







#---------------------------------------------------------------------------------------------------------------------


def band_pass_hsv(bgr_color):
	"""given a color in bgr colorspace, returns the min and max values for the bandpass filter"""
	hsv_color = cv2.cvtColor(bgr_color.reshape(1, 1, 3), cv2.COLOR_BGR2HSV)[0][0]
	hsv_min = hsv_color - TOLERANCE_HSV
	hsv_max = hsv_color + TOLERANCE_HSV

	return hsv_min, hsv_max


#---------------------------------------------------------------------------------------------------------------------


def filter_hsv_to_binary_old(hsv_img, hsv_min, hsv_max, binary_threshold):
	"""transform a HSV image to a binary image with threshold of the desired color
	| hsv_img : image in the hsv colorspace
	| hsv_min, hsv_max : min and max of color filter
	| binary_threshold : threshold under which the pixel is 0 and over which the pixel is 255"""
	#keep only the desired color in hsv
	mask = cv2.inRange(hsv_img, hsv_min, hsv_max)
	output = cv2.bitwise_and(hsv_img,hsv_img, mask= mask)

	grayscale = output[:, :, 2]
	#convert to binary
	threshold = binary_threshold
	ret, bw = cv2.threshold(grayscale, threshold, 255, cv2.THRESH_BINARY)
	return bw


#----------------------------------------------------------------------------------------------------------------------------------


def filter_to_hsv(image):
	"""Filters the image from BGR colorspace to HSV colorspace"""
	filtered_img = cv2.GaussianBlur(image,(5,5),1)
    
    # Convert BGR to HSV
	hsv_img = cv2.cvtColor(filtered_img, cv2.COLOR_BGR2HSV)
	return hsv_img


#----------------------------------------------------------------------------------------------------------------------------------


def filter_hsv_to_binary(hsv_img, bgr_color, binary_threshold):
	"""filters the color of an hsv image to a binary image
	| hsv img : image in the hsv colorspace
	| bgr_color : color to filter in the bgr colorspace
	| binary threshold : value btw 0 and 255 for binary filter"""
	hsv_min, hsv_max = band_pass_hsv(bgr_color)
	mask = cv2.inRange(hsv_img, hsv_min, hsv_max)
	output = cv2.bitwise_and(hsv_img,hsv_img, mask= mask)

	grayscale = output[:, :, 2]
	#convert to binary
	threshold = binary_threshold
	ret, bw = cv2.threshold(grayscale, threshold, 255, cv2.THRESH_BINARY)
	return bw


#------------------------------------------------------------------------------------------------------------------------------------


def filter_grayscale(img):
	"given image in bgr colorspace, convert it to grayscale then convert it to binary"
	#convert to grayscale both image and template
	img_grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

	#convert to binary both image and template
	threshold = BINARY_THRESHOLD_BLACK
	ret, image_binary = cv2.threshold(img_grayscale, threshold, 255, cv2.THRESH_BINARY)

	return image_binary
	

#--------------------------------------------------------------------------------------------------------------------------------------


def non_max_suppression_fast(boxes, overlapThresh):
	"""Makes sure to only have one set of coordinate per cross to find the corners of the map"""
	# if there are no boxes, return an empty list
	if len(boxes) == 0:
		return []
	# if the bounding boxes integers, convert them to floats --
	# this is important since we'll be doing a bunch of divisions
	if boxes.dtype.kind == "i":
		boxes = boxes.astype("float")
	# initialize the list of picked indexes	
	pick = []
	# grab the coordinates of the bounding boxes
	x1 = boxes[:,0]
	y1 = boxes[:,1]
	x2 = boxes[:,2]
	y2 = boxes[:,3]
	# compute the area of the bounding boxes and sort the bounding
	# boxes by the bottom-right y-coordinate of the bounding box
	area = (x2 - x1 + 1) * (y2 - y1 + 1)
	idxs = np.argsort(y2)
	# keep looping while some indexes still remain in the indexes
	# list
	while len(idxs) > 0:
		# grab the last index in the indexes list and add the
		# index value to the list of picked indexes
		last = len(idxs) - 1
		i = idxs[last]
		pick.append(i)
		# find the largest (x, y) coordinates for the start of
		# the bounding box and the smallest (x, y) coordinates
		# for the end of the bounding box
		xx1 = np.maximum(x1[i], x1[idxs[:last]])
		yy1 = np.maximum(y1[i], y1[idxs[:last]])
		xx2 = np.minimum(x2[i], x2[idxs[:last]])
		yy2 = np.minimum(y2[i], y2[idxs[:last]])
		# compute the width and height of the bounding box
		w = np.maximum(0, xx2 - xx1 + 1)
		h = np.maximum(0, yy2 - yy1 + 1)
		# compute the ratio of overlap
		overlap = (w * h) / area[idxs[:last]]
		# delete all indexes from the index list that have
		idxs = np.delete(idxs, np.concatenate(([last],
			np.where(overlap > overlapThresh)[0])))
	# return only the bounding boxes that were picked using the
	# integer data type
	return boxes[pick].astype("int")


#----------------------------------------------------------------------------------------------------------------------------------


def order_points(pts):
	"""Orders points of rectangle so that they are in the order : top_left, top_right, bottom_right, bottom_left """
	# sort the points based on their x-coordinates
	xSorted = pts[np.argsort(pts[:, 0]), :]
	# grab the left-most and right-most points from the sorted
	# x-roodinate points
	leftMost = xSorted[:2, :]
	rightMost = xSorted[2:, :]
	# now, sort the left-most coordinates according to their
	# y-coordinates so we can grab the top-left and bottom-left
	# points, respectively
	leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
	(tl, bl) = leftMost
	# now that we have the top-left coordinate, use it as an
	# anchor to calculate the Euclidean distance between the
	# top-left and right-most points; by the Pythagorean
	# theorem, the point with the largest distance will be
	# our bottom-right point
	D = dist.cdist(tl[np.newaxis], rightMost, "euclidean")[0]
	(br, tr) = rightMost[np.argsort(D)[::-1], :]
	# return the coordinates in top-left, top-right,
	# bottom-right, and bottom-left order
	return np.array([tl, tr, br, bl], dtype="float32")


#--------------------------------------------------------------------------------------------------------------------------------------------------


def find_4_corners(img, template, method):
	"""Finds the coordiantes of the corners of the map; 	
	arguments :	
	| img : image in the bgr color space;	
	| template : image that has been filtered and is in the binary color space;	
	| method : cross-correlation method for template matching
	"""
	is_found = False
	
	#filter the image
	filtered_img = cv2.GaussianBlur(img,(5,5), 1)
	
	#convert image and template to binary
	image_binary = filter_grayscale(filtered_img)
	template_binary = filter_grayscale(template)
	
	cv2.imshow("binary of black", image_binary)
	#get template dimensions
	c, w, h  = template.shape[::-1]
	#print(template.shape[::-1])
	
	#match template to image
	res = cv2.matchTemplate(image_binary,template_binary,method)
	(yCoords, xCoords) = np.where(res >= RES_THRESHOLD_BLACK)


	# initialize our list of rectangles
	rects = []
	# loop over the starting (x, y)-coordinates again
	for (x, y) in zip(xCoords, yCoords):
		# update our list of rectangles
		rects.append((x, y, x + w, y + h))
	
	# apply non-maxima suppression to the rectangles
	pick = non_max_suppression_fast(np.array(rects), OVERLAP_THRESHOLD)
	#print("[INFO] {} matched locations after NMS".format(len(pick)))
	
	#create a list of centers of the rectangles
	cross_points = []
	for (startX, startY, endX, endY) in pick:
	    # draw the bounding box on the image
		cv2.rectangle(image_binary, (startX, startY), (endX, endY), (0, 0, 255), 3)
		cross_points.append([int((startX+endX)/2), int((startY+ endY)/2)]) 

	#cv2.imshow("binary of black", image_binary)
	#cv2.imshow("template", template_binary)

	if (len(pick) == 4) :
		is_found = True
	
	cross_points = np.array(cross_points)
	cv2.imwrite("template_cross_binary.png", image_binary)
	
	return is_found, cross_points



#----------------------------------------------------------------------------------------------------------------------------------


def warp_image(image, template, method, old_corners):
	"""Transforms the camera image to a map bird's eye view with only the map"""
	is_found, cross_points = find_4_corners(image, template, method)
	
	if is_found:
		ordered_crosses = order_points(cross_points)
		#store corners positions in case we have a frame where we don't see them anymore
		old_corners = ordered_crosses
	else : 
		#will become the last found corners, 
		ordered_crosses = old_corners

	ordered_crosses = np.float32(ordered_crosses)
	bird_corners = np.float32(BIRD_CORNERS)

	matrix = cv2.getPerspectiveTransform(ordered_crosses, bird_corners)
	bird_image = cv2.warpPerspective(image, matrix, (BIRD_WIDTH, BIRD_HEIGHT))

	return is_found, bird_image, old_corners


#----------------------------------------------------------------------------------------------------------------------------------


def create_obstacles(hsv_image):
	"""Finds the obstacles (orange parts) on the map and put them on the map"""
	#filter the orange color
	mask = cv2.inRange(hsv_image, HSV_ORANGE_MIN, HSV_ORANGE_MAX)
	output = cv2.bitwise_and(hsv_image,hsv_image, mask= mask)

	#convert map to binary (0 = no obstacle, 1 = obstacle)
	output_gray = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)
	threshold = 27
	ret, map = cv2.threshold(output_gray, threshold, 255, cv2.THRESH_BINARY)

	#map[map==255] = OBSTACLE_MAP_VALUE
	return map


#------------------------------------------------------------------------------------------------------------------------------------


def dilate_obstacles(map, width):
	"""Dilates the obstacles to approximate the size of the thymio to one pixel
	map : [array] of 0 (background) and 1 (obstacles)
	width : [int] half the width of the thymio in pixels"""

	kernel = np.ones((2*width + 1, 2*width + 1), np.uint8)
	eroded_image = cv2.erode(map, kernel, iterations = 1)
	dilated_img = cv2.dilate(eroded_image, kernel, iterations = 2)

	return dilated_img

#---------------------------------------------------------------------------------------------------------------------------------------


def create_map_borders(map, width):
	"""put the borders of the map as obstacles"""
	map[0:width, :BIRD_WIDTH] = 255
	map[:BIRD_HEIGHT, BIRD_WIDTH-width:BIRD_WIDTH] = 255
	map[BIRD_HEIGHT-width: BIRD_HEIGHT, :BIRD_WIDTH] = 255
	map[:BIRD_HEIGHT, 0:width] = 255
	return map


#-----------------------------------------------------------------------------------------------------------------------------------------


def create_map(hsv_image):
	"""take the bird's eye image and convert it to a map, without the goal coordinates"""
	obstacles = create_obstacles(hsv_image)
	dilated_map = dilate_obstacles(map, THYMIO_WIDTH)
	map = create_map_borders(obstacles, THYMIO_WIDTH)
	map[map == 255] = OBSTACLE_MAP_VALUE
	return dilated_map



#------------------------------------------------------------------------------------------------------------------------------------


def get_goal_coord(hsv_img, template, method):
	"""Finds the position of the goal in the map, returns corners of rectangle and position
	| hsv_img : image in the hsv colorspace
	| template : template in binary colorspace of the goal
	| method : the cross correlation method (see exercises week 3)"""
	#keep only the green (color of goal)
	bw = filter_hsv_to_binary(hsv_img, GREEN_BGR, BINARY_THRESHOLD_GREEN)
	template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
	ret, template_bw = cv2.threshold(template_gray, BINARY_THRESHOLD_GREEN, 255, cv2.THRESH_BINARY)

	#find template coordinates
	res = cv2.matchTemplate(bw, template_bw, method)
	min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
	#get template dimensions
	c, w, h  = template.shape[::-1]

	#determine if we have found the coordinates or if it might be a fake
	if max_val >= RES_THRESHOLD_RED:
		is_found = True
		top_left = max_loc
		bottom_right = (top_left[0] + w, top_left[1] + h)
		position = (top_left[0] + int(w/2), top_left[1] + int(h/2))
	else:
		top_left = (0, 0)
		bottom_right = (w, h)#set the thymio position to the top left corner if we haven't found it
		position = (0, 0)

	cX, cY = position
	position = (cY, cX) #invert positions to make the axis like those of a np array
	return is_found, top_left, bottom_right, position


#----------------------------------------------------------------------------------------------------------------------


def find_l_thymio(bw, template, method):
	"""Finds coordinates of thymio
	arguments :	
	| bw : image in the binary color space;	
	| template : image that has been filtered and is in the binary color space (1 dim);	
	| method : cross-correlation method for template matching
	"""
	is_found = False
	w, h  = template.shape[::-1]
	
	cv2.imshow("red filter", bw)
	cv2.imwrite("red_filter.jpg", bw)
	

	#find location of thymio
	contours, hierarchy  = cv2.findContours(bw, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

	#print(len(contours), "elements found")
	if len(contours) != 0:
		#if contour detected, keep the biggest one (most likely the thymio)
		contours = sorted(contours, key = cv2.contourArea, reverse = True)
		contour = contours[0]
		#print("area of best contour rn", cv2.contourArea(contour))

		contour_threshold = 100 #detect if contour found is too small
		M = cv2.moments(contour)
		if (M["m00"] != 0) and (cv2.contourArea(contour) >= contour_threshold) :
			cX = int(M["m10"]/M["m00"])
			cY = int(M["m01"]/M["m00"])
			is_found = True
			position = (cX, cY)
		else :
			#division by zero or no contour big enough found
			#print(M["m00"], "contour too small") 
			position = (0, 0)
			is_found = False
	else:
		#no contour found
		#print("no contours found")
		position = (0, 0)
		is_found = False
	
		

	return is_found, position


#----------------------------------------------------------------------------------------------------------------------


def find_thymio_angle(image, template, method, position):
	"""Finds the orientation of the thymio
	| image : binary image of the map, where we can do template matching
	| template : binary image of the sticker on top of the thymio (be careful of thymio's leds)
	| method : type of cross correlation (see exercise week 3)
	| position : position of the thymio"""
	w, h = template.shape[::-1]
	template_center = (TEMP_CENTER_X, TEMP_CENTER_Y)
	
	thymio_loc = image[position[1] - TEMP_CENTER_Y : position[1] + (h-TEMP_CENTER_Y), position[0] - TEMP_CENTER_X: position[0] + (w-TEMP_CENTER_X)]
	#for each angle, rotate template and compare it to what is in the picture.
	angle_scores = np.zeros(360)
	for i in range(360):
		rotation_mat = cv2.getRotationMatrix2D(template_center, i, 1.0)
		rotated = cv2.warpAffine(template, rotation_mat, (w, h))
		angle_scores[i] = cv2.matchTemplate(rotated, thymio_loc, method)[0][0]

	#the angle of the thymio is the one that has the highest correlation
	best_angle = np.argmax(angle_scores)
	value = angle_scores.max()
	#print("cross corr value", value)
	return best_angle, value


#------------------------------------------------------------------------------------------------------------------------------


def get_thymio_pose(hsv_img, template, method):
	"""returns position and angle of thymio, and a boolean that is true if thymio has been found
	| hsv_img : image in the hsv colorspace
	| template : template in binary colorspace of the sticker on top of the thymio
	| method : the cross correlation method (see exercises week 3)"""
	#keep only red and filter to binary
	bw = filter_hsv_to_binary(hsv_img, RED_BGR, BINARY_THRESHOLD_RED)
	
	#convert template to binary
	template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
	threshold = BINARY_THRESHOLD_RED
	ret2, template_binary = cv2.threshold(template, threshold, 255, cv2.THRESH_BINARY)
	
	#find position of thymio
	is_found, position = find_l_thymio(bw, template_binary, method)
	
	if is_found:
		#print("position before the angle", position)
		angle, value = find_thymio_angle(bw, template_binary, method, position)
		
		if value <= RES_THRESHOLD_RED:
			print("angle cross corr value is too low")
			is_found = False
			angle = 0
			position = (0, 0)
	else:
		angle = 0
		position = (0, 0)

	print("position", is_found, position, angle)
	#print("angle", angle)
	cX, cY = position
	position = (cY, cX) #invert positions to make the axis like those of a np array
	return is_found, position, angle #for now, the position is a pixel location, will need to convert it later


#--------------------------------------------------------------------------------------------------------------------------------------------------------


def make_sure_corners_are_found(image, template, method):
	"""Function to calibrate the camera and make sure we can detect all the corners"""
	is_found, cross_points = find_4_corners(image, template, method)
	print("The corners were found :", is_found)
	image_copy = image.copy()
	for point in cross_points:
		cv2.circle(image_copy, tuple(point), 20, (0, 0, 255), 3)
	return is_found
	

#-------------------------------------------------------------------------------------------------------------------------------------------


def add_goal_to_map(map, goal_position): #probably won't use it
	map[goal_position] = 2 #value to change for now
	return map


#-------------------------------------------------------------------------------------------------------------------------------------------


def add_start_to_map(map, position): #probably won't use it
	#might want to add a condition to be sure thymio is not on an obstacle 
	map[position] = 3 #value to change for now
	return map


#--------------------------------------------------------------------------------------------------------------------------------------------------


def wait_for_cam_to_calibrate():
	while True:
		time.sleep(0.1)
		if cv2.waitKey(1) == ord('s'):
			break



#----------------------------------------------------------------------------------------------------------------------------------------------------


def is_camera_obstructed(image):

    img_grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

	#convert to binary both image and template
    threshold = 180
    
    ret, image_binary = cv2.threshold(img_grayscale, threshold, 255, cv2.THRESH_BINARY)
    
    

    zero_points = cv2.findNonZero(image_binary)
    
    if (zero_points is None) or (len(zero_points) < 200):
        is_obstructed = True
    else:
        is_obstructed = False
    return is_obstructed


#----------------------------------------------------------------------------------------------------------------------------------------------------------------



# Open the default camera
cam = cv2.VideoCapture(2)

if not cam.isOpened():
    print("Error: Could not open camera.")


#convert camera resolution from 1920x1080 to 1024x768
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1024)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 768)


#set it for later
old_corners = CAMERA_CORNERS #variable that will be updated each time



while True:
    
	ret, image = cam.read()
	image_copy = image.copy()

	#cv2.imshow("Camera", image_copy)
	#warp image
	corners_found, bird_image, old_corners = warp_image(image, template_cross, method, old_corners)

	#convert to hsv colorspace
	hsv_img = filter_to_hsv(bird_image)

	map = create_obstacles(hsv_img)
	cv2.imshow("map before", map)
	map = dilate_obstacles(map, 5)

	
	is_thymio_found, thymio_position, angle = get_thymio_pose(hsv_img, template_l_thymio, method)


	

	# Display the captured frame
	#cv2.imshow('Camera', image_copy)
	cv2.imshow("Bird's eye view", bird_image)
	#cv2.imshow("template_thymio", template_l_thymio)
	cv2.imshow("map", map)
 
    
	# Press 'q' to exit the loop
	if cv2.waitKey(1) == ord('q'):
		cv2.imwrite("map_in_polydome2.jpg", map)
		break


# Release the capture and writer objects
cam.release()
cv2.destroyAllWindows()